In [ ]:
# المكتبات الأساسية
import os
import requests
import subprocess

# إعداد اسم المستخدم الافتراضي
default_github_username = "TamerOnLine"

# الحصول على اسم المستخدم من المستخدم أو استخدام الافتراضي
github_username = input(f"Enter your GitHub username (default: {default_github_username}): ") or default_github_username

# الحصول على اسم ملف الـ ipynb الحالي بدون الامتداد
notebook_name = os.path.splitext(os.path.basename(os.getcwd()))[0]

# إعداد رابط المستودع على GitHub باستخدام اسم المشروع واسم المستخدم
github_url = f"https://github.com/{github_username}/{notebook_name}.git"

# التحقق من وجود مستودع Git في المجلد الحالي، وإن لم يكن موجودًا، يتم تهيئته
try:
    # التحقق من وجود مجلد .git
    subprocess.run(["git", "rev-parse", "--is-inside-work-tree"], check=True, capture_output=True)
    print("Git repository already initialized.")
except subprocess.CalledProcessError:
    # تهيئة مستودع جديد إذا لم يكن موجوداً
    print("Git repository not found. Initializing a new Git repository...")
    subprocess.run(["git", "init"], check=True)

# أمر Git للتحقق من إعداد remote origin أو إضافته إذا لم يكن موجوداً
if subprocess.run(["git", "remote", "get-url", "origin"], capture_output=True, text=True).returncode != 0:
    subprocess.run(["git", "remote", "add", "origin", github_url], check=True)
    print(f"Remote repository set to: {github_url}")

# سحب آخر التحديثات من الفرع الرئيسي (تجنب الخطأ عند عدم وجود المستودع)
try:
    subprocess.run(["git", "pull", "origin", "main"], check=True)
    print("Pulled latest updates from the main branch.")
except subprocess.CalledProcessError:
    print("Error: Could not pull updates. Make sure the repository exists and is accessible.")

# معالجة تحويل التنسيقات بين LF و CRLF
try:
    subprocess.run(["git", "add", "."], check=True)
except subprocess.CalledProcessError:
    print("Error: Could not add files. Check if the file paths are correct.")

# إعداد رسالة الالتزام (commit) مع إمكانية تغيير الرسالة حسب الحاجة
commit_message = "Initial commit for project setup"
try:
    subprocess.run(["git", "commit", "-m", commit_message], check=True)
    print(f"Commit successful with message: {commit_message}")
except subprocess.CalledProcessError:
    print("Error: Could not commit changes. Make sure there are changes to commit.")

# إعداد الفرع الرئيسي (main) كمكان الالتزامات الافتراضي
subprocess.run(["git", "branch", "-M", "main"], check=True)

# محاولة رفع التحديثات إلى GitHub (معالجة الأخطاء)
try:
    subprocess.run(["git", "push", "-u", "origin", "main"], check=True)
    print("Pushed updates to GitHub successfully.")
except subprocess.CalledProcessError:
    print("Error: Could not push updates. Check your remote repository configuration.")

# التحقق من وجود المستودع على GitHub باستخدام API
response = requests.get(f"https://api.github.com/repos/{github_username}/{notebook_name}")

if response.status_code == 200:
    print(f"Repository '{notebook_name}' found on GitHub for user '{github_username}'.")
else:
    print(f"Repository '{notebook_name}' not found on GitHub.")
    print(f"Please create the repository '{notebook_name}' on GitHub first, then try again.")
    print(f"Suggested repository name: {notebook_name}")
